# 1.

In [37]:
import pandas as pd
import datetime

mldb = pd.read_csv('ml1mdata2000.csv')
mldb.isna().sum()

user_id        13
movie_id       80
rating          0
timestamp       0
gender          0
age            66
occupation     55
zip            73
title         173
genres         93
dtype: int64

# 2.

In [30]:
# add missing column and assign 0 if no missing values, 1 if there are missing values
mldb["missing"] = 0
mldb.loc[mldb.isna().any(axis=1), "missing"] = 1
# show records with missing values
mldb[mldb.isna().any(axis=1)]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,missing
8,28.0,1193.0,3,978125194,F,25.0,1.0,14607,One Flew Over the Cuckoo's Nest (1975),NaN,1
9,33.0,1193.0,5,978557765,M,45.0,3.0,55421,One Flew Over the Cuckoo's Nest (1975),NaN,1
10,39.0,1193.0,5,978043535,M,18.0,4.0,61820,One Flew Over the Cuckoo's Nest (1975),NaN,1
11,42.0,1193.0,3,978038981,M,25.0,8.0,24502,One Flew Over the Cuckoo's Nest (1975),NaN,1
12,44.0,1193.0,4,978018995,M,45.0,17.0,98052,One Flew Over the Cuckoo's Nest (1975),NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
1000162,4486.0,133.0,1,965013057,M,35.0,8.0,98674,NaN,Comedy|Drama,1
1000163,4553.0,3290.0,3,964551353,F,35.0,16.0,48237,NaN,Comedy,1
1000164,4553.0,1709.0,2,964551353,F,35.0,16.0,48237,NaN,Thriller,1
1000165,4572.0,3164.0,4,964460301,F,1.0,10.0,17036,NaN,Drama,1


In [31]:
# create df with movie_id and the number of records of that movie_id
mldb1 = mldb.groupby(["movie_id"])["movie_id"].agg("count").reset_index(name="num_records")
mldb1

,movie_id,num_records
0,1.0,2077
1,2.0,701
2,3.0,478
3,4.0,170
4,5.0,296
...,...,...
3700,3948.0,862
3701,3949.0,304
3702,3950.0,54
3703,3951.0,40


In [32]:
# create df with movie_id and number of missing records for that movie_id
mldb2 = mldb.groupby(["movie_id"])["missing"].agg("sum").reset_index(name="num_missing")
mldb2

,movie_id,num_missing
0,1.0,0
1,2.0,0
2,3.0,0
3,4.0,0
4,5.0,0
...,...,...
3700,3948.0,0
3701,3949.0,0
3702,3950.0,0
3703,3951.0,0


In [33]:
# join two df and create column counting % missing values for each movie_id
mldb_info = pd.merge(mldb1, mldb2, how="outer")
mldb_info["missing_percent"] = mldb_info["num_missing"]/mldb_info["num_records"]
mldb_info

,movie_id,num_records,num_missing,missing_percent
0,1.0,2077,0,0.0
1,2.0,701,0,0.0
2,3.0,478,0,0.0
3,4.0,170,0,0.0
4,5.0,296,0,0.0
...,...,...,...,...
3700,3948.0,862,0,0.0
3701,3949.0,304,0,0.0
3702,3950.0,54,0,0.0
3703,3951.0,40,0,0.0


In [36]:
# filter to only show movie_ids that have missing values
mldb_info[mldb_info["num_missing"] > 0]

,movie_id,num_records,num_missing,missing_percent
90,93.0,74,8,0.108108
128,133.0,1,1,1.000000
343,353.0,734,51,0.069482
354,364.0,1121,11,0.009813
379,389.0,20,7,0.350000
816,874.0,8,6,0.750000
853,914.0,636,52,0.081761
1093,1181.0,10,6,0.600000
1095,1184.0,91,11,0.120879
1101,1190.0,132,11,0.083333


# 3.

In [44]:
# turn timestamp into a date
mldb['date']= pd.to_datetime(mldb['timestamp'], unit='s', errors='coerce')

# turn date into month
mldb['month'] = pd.DatetimeIndex(mldb['date']).month

# find records from final quarter, month > 9
finalQ = mldb.loc[mldb.month > 9]
finalQ

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,month,date
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-31 22:12:40
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-31 21:33:33
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-30 23:49:39
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-30 18:01:19
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-30 06:41:11
...,...,...,...,...,...,...,...,...,...,...,...,...
1000139,4169.0,701.0,4,976588990,M,50.0,0.0,66048,Daens (1992),Drama,12,2000-12-12 02:43:10
1000149,4169.0,138.0,4,975804273,M,50.0,0.0,66048,"Neon Bible, The (1995)",Drama,12,2000-12-03 00:44:33
1000151,4169.0,530.0,4,976589311,M,50.0,0.0,66048,Second Best (1994),Drama,12,2000-12-12 02:48:31
1000153,4169.0,1070.0,3,971580375,M,50.0,0.0,66048,Macao (1952),Adventure,10,2000-10-15 03:26:15


# 4.

In [45]:
# create new column using rating category to create levels of ratings
mldb["rating_category"]= pd.cut(mldb["rating"], bins=[0,2,4,5], labels=['ok', 'good', 'excellent'])
mldb

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,month,date,rating_category
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-31 22:12:40,excellent
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-31 21:33:33,excellent
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-30 23:49:39,good
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-30 18:01:19,good
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,12,2000-12-30 06:41:11,excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary,5,2000-05-20 18:13:21,excellent
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama,12,2000-12-05 15:11:56,good
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama,5,2000-05-12 17:37:48,NaN
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western,5,2000-05-08 03:30:08,excellent


# 5.

In [53]:
# create new movie ranking df with mean rating column
movie_rank = mldb.groupby(["movie_id"])["rating"].mean().reset_index(name = "avg_rating")

# create a rank column using average rating to rank the movies
movie_rank["rank"] = movie_rank["avg_rating"].rank(ascending = False)

# create a percentile column using the rank column
movie_rank["percentile"] = movie_rank["avg_rating"].rank(pct = True)

movie_rank

,movie_id,avg_rating,rank,percentile
0,1.0,4.146846,203.0,0.945479
1,2.0,3.201141,2117.0,0.428880
2,3.0,3.016736,2440.0,0.341700
3,4.0,2.729412,2914.0,0.213765
4,5.0,3.006757,2453.0,0.338192
...,...,...,...,...
3700,3948.0,3.635731,1168.0,0.685020
3701,3949.0,4.115132,231.0,0.937922
3702,3950.0,3.666667,1099.5,0.703509
3703,3951.0,3.900000,592.0,0.840486


# 6.

In [64]:
# new df drop records with missing values in zip column
temp = mldb.dropna(subset=["zip"])

# new df show movies containg 007 in zip
oo7 = temp[temp.zip.str.contains("007")]

# group by movie_id and show mean rating
oo7.groupby(["movie_id"])["rating"].mean().reset_index(name = "avg_rating")

,movie_id,avg_rating
0,1.0,4.142857
1,2.0,3.000000
2,3.0,2.400000
3,4.0,2.000000
4,5.0,3.000000
...,...,...
1569,3928.0,4.000000
1570,3930.0,4.000000
1571,3946.0,2.000000
1572,3948.0,4.333333
